### Graph Neural Networks

Graph Neural Networks on the Lund Plane in Julia! 

TODO:
 - Run on GPU machine using CUDA [x]
 - Try to parallelise pre-procesing step on GPU02 -- there are 32 CPU cores [x]
 - Implement a new GNN layer that performs CA re-clustering after EdgeConv operation

In [1]:
#To install everything on a new machine
# import Pkg
# Pkg.add(["Flux","Statistics","StatsBase","GraphNeuralNetworks", "CUDA", 
#          "Graphs","MLUtils","Plots","GraphMakie","CairoMakie", "UnROOT", "Trapz" ])

In [2]:
# using ProgressMeter
using Flux, Statistics, StatsBase, GraphNeuralNetworks, Graphs, MLUtils, Plots, CUDA
using Flux.Data: DataLoader
using GraphMakie, CairoMakie
using Trapz, Printf
using UnROOT #Julia native package for reading root files
import Interpolations
using Random
using Glob
using BSON

In [3]:
include("lundplane_utils.jl")

load_chunk_from_rootfiles (generic function with 2 methods)

In [4]:
#Check how many threads we're running on
# Threads.nthreads()

### Open LJP data 

In [5]:
# # f_sig = ROOTFile("../Downloads/ntuples_test/wprime/user.asopio.24603642._000001.ANALYSIS.root")
# # f_bkg = ROOTFile("../Downloads/ntuples_test/dijets/user.asopio.24603630._000002.ANALYSIS.root")
# f_sig = ROOTFile("/unix/atlas2/asopio/lundNtuples_20210322/user.asopio.grid_wprime_trees_ufosd.426347.Pythia8EvtGen_A14NNPDF23LO_WprimeWZ_flatpT_ANALYSIS.root/user.asopio.24603642._000001.ANALYSIS.root")
# f_bkg = ROOTFile("/unix/atlas2/asopio/lundNtuples_20210322/user.asopio.grid_dijet_trees_ufosd.364706.Pythia8EvtGen_A14NNPDF23LO_jetjet_JZ6WithSW_ANALYSIS.root/user.asopio.24603630._000002.ANALYSIS.root")

# brkeys = ["Akt10TruthChargedJet_jetLundZ", "Akt10TruthChargedJet_jetLundDeltaR", "Akt10TruthChargedJet_jetLundKt", 
#           "Akt10TruthChargedJet_jetLundPt1", "Akt10TruthChargedJet_jetLundPt2", "Akt10TruthChargedJet_jetLundE",
#           "Akt10TruthJet_jetM", "Akt10TruthJet_jetPt", "Akt10TruthChargedJet_jetLundIDParent1", 
#           "Akt10TruthChargedJet_jetLundIDParent2"]

# tname = "lundjets_InDetTrackParticles"

# t_sig = LazyTree(f_sig, tname, brkeys)
# t_bkg = LazyTree(f_bkg, tname, brkeys)

# ""

In [6]:
# t_sig[1].Akt10TruthJet_jetM[1]

In [7]:
# nsamp = 10000

# t = @elapsed begin

# sig_samples = make_graphs_from_ttree( t_sig, nsamp, lab=1 )
# bkg_samples = make_graphs_from_ttree( t_bkg, nsamp, lab=0 )
    
# end
# println( "Seconds to generate dataset graphs ", t )

In [8]:
device = CUDA.functional() ? Flux.gpu : Flux.cpu;

In [9]:
model_name = "model_bsons/myGNN_6LPfeats_v3"

# device = Flux.cpu;
model = GNNChain( EdgeConv( Dense(12,64) ), #GCNConv(16 => 64), #EdgeConv( Dense(16,16) ), 
#                   BatchNorm(64),     # Apply batch normalization on node features (nodes dimension is batch dimension)
                  x -> relu.(x),
                  EdgeConv( Dense(128,64) ),
                  x -> relu.(x),
                  EdgeConv( Dense(128,128) ),
                  x -> relu.(x),
                  #GCNConv(64 => 64, relu),
                  GlobalPool(mean),  # aggregate node-wise features into graph-wise features
                  Dense(128, 256), 
                  Dense(256, 256), 
                  Dense(256, 1), 
                  x -> sigmoid(x) )#Sigmoid since we need values in [0-1]

ps = Flux.params(model)
# opt = Adam(1f-4)
opt = Flux.Optimise.Adam(1f-4)
# opt = Flux.Optimise.Descent(1f-4)

Adam(9.999999747378752e-5, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

In [10]:
sum(length, ps)

124673

In [11]:
outfname = string(model_name,"_0.bson")
print("Saving model ", outfname)
BSON.@save outfname model

Saving model model_bsons/myGNN_6LPfeats_v3_0.bson

In [12]:
#Load the latest epoch of the model
saved_models = glob(model_name * "*")
max_saved_epoch = maximum([ parse(Int64,split(m,"_")[end][1:end-5]) for m in saved_models ])
println("Loading model ", "$(model_name)_$(max_saved_epoch).bson")
BSON.@load "$(model_name)_$(max_saved_epoch).bson" model

Loading model model_bsons/myGNN_6LPfeats_v3_11.bson


In [13]:
model

GNNChain(EdgeConv(Dense(12 => 64), aggr=max), #6, EdgeConv(Dense(128 => 64), aggr=max), #7, EdgeConv(Dense(128 => 128), aggr=max), #8, GlobalPool{typeof(mean)}(Statistics.mean), Dense(128 => 256), Dense(256 => 256), Dense(256 => 1), #9)

In [14]:
# CUDA.reclaim()

In [ ]:
loss(g::GNNGraph) = mean((vec(model(g, g.ndata.x)) - g.gdata.y).^2)
loss(loader) = mean(loss(g |> device) for g in loader)

#Number of chunks        
nch = 50

#globs for signal/background samples
gl_sig = "../lundNtuples_20210322/user.asopio.grid_wprime*_ANALYSIS.root/*"
gl_bkg = "../lundNtuples_20210322/user.asopio.grid_dijet*_ANALYSIS.root/*"
        
#Batch size for training
# bs=1024 * 2
bs=128
        
test_losses = Float32[ 999.0 ]
train_losses = Float32[ 999.0 ]

nepochs = 25
prev_train_loss = 999
min_epoch = maximum((1,max_saved_epoch))
        
t_ep = 0.0
t_ch = 0.0

#load the model onto the device        
model = device(model) 
        
println("Training on ", nch, " chunks")
for epoch in min_epoch:nepochs
    t_ep = @elapsed begin
        for ich in 1:nch                    
            t_ch = @elapsed begin  
                sig_samples = load_chunk_from_rootfiles( gl_sig, nch, ich, 1.0 )
                bkg_samples = load_chunk_from_rootfiles( gl_bkg, nch, ich, 0.0 )

                all_samples = GNNGraph[]
                append!( all_samples, sig_samples )
                append!( all_samples, bkg_samples )

                Random.seed!(1234)
                train_graphs, test_graphs = MLUtils.splitobs(all_samples, at=0.99, shuffle=true)

                train_loader = DataLoader(train_graphs, 
                                batchsize=bs, shuffle=true, collate=true)
                test_loader = DataLoader(test_graphs, 
                                batchsize=bs, shuffle=false, collate=true)

#                 CUDA.@time begin
                            
#                 batch_train_losses = Float32[]        
                #Train loop
                for g in train_loader
                    g = g |> device
                    grad = Flux.gradient(() -> loss(g), ps)
                    Flux.Optimise.update!(opt, ps, grad)
                end
                
                train_loss = loss( train_loader ) #Only evaluate losses on first batch for speed
                test_loss = loss( test_loader )
                        
#                 train_loss = loss( first(train_loader) |> device ) #Only evaluate losses on first batch for speed
#                 test_loss = loss( first(test_loader) |> device )
                            
#                 end
                            
                push!( train_losses, train_loss )
                push!( test_losses, test_loss )
                        
                @info (; epoch, ch=ich, train_loss=train_loss, test_loss=test_loss, 
                         dloss=train_losses[end-1] - train_loss, seconds_per_chunk=t_ch, seconds_per_epoch=t_ep)
                        
            end
        end            
    end
            
    outfname = string(model_name,"_",epoch,".bson")
    print("Saving model ", outfname)
    let model = cpu(model)
        BSON.@save outfname model        
    end
end

Training on 50 chunks


[ Info: (epoch = 11, ch = 1, train_loss = 0.2228989f0, test_loss = 0.21674263f0, dloss = 998.7771f0, seconds_per_chunk = 0.0, seconds_per_epoch = 0.0)
[ Info: (epoch = 11, ch = 2, train_loss = 0.22294189f0, test_loss = 0.22145145f0, dloss = -4.298985f-5, seconds_per_chunk = 119.203391926, seconds_per_epoch = 0.0)
[ Info: (epoch = 11, ch = 3, train_loss = 0.2225648f0, test_loss = 0.22487965f0, dloss = 0.00037708879f0, seconds_per_chunk = 22.583294814, seconds_per_epoch = 0.0)
[ Info: (epoch = 11, ch = 4, train_loss = 0.22227429f0, test_loss = 0.22642002f0, dloss = 0.00029051304f0, seconds_per_chunk = 22.956784675, seconds_per_epoch = 0.0)
[ Info: (epoch = 11, ch = 5, train_loss = 0.2231141f0, test_loss = 0.22305323f0, dloss = -0.00083981454f0, seconds_per_chunk = 24.477878922, seconds_per_epoch = 0.0)
[ Info: (epoch = 11, ch = 6, train_loss = 0.20838696f0, test_loss = 0.2082473f0, dloss = 0.014727145f0, seconds_per_chunk = 23.153383135, seconds_per_epoch = 0.0)
[ Info: (epoch = 11, ch =

Saving model model_bsons/myGNN_6LPfeats_v3_11.bson

[ Info: (epoch = 11, ch = 50, train_loss = 0.18335491f0, test_loss = 0.18858157f0, dloss = 5.4866076f-5, seconds_per_chunk = 104.988067692, seconds_per_epoch = 0.0)
[ Info: (epoch = 12, ch = 1, train_loss = 0.22290182f0, test_loss = 0.21674263f0, dloss = -0.039546907f0, seconds_per_chunk = 89.020010307, seconds_per_epoch = 3200.828148853)
[ Info: (epoch = 12, ch = 2, train_loss = 0.2229419f0, test_loss = 0.22145148f0, dloss = -4.0084124f-5, seconds_per_chunk = 22.692477723, seconds_per_epoch = 3200.828148853)
[ Info: (epoch = 12, ch = 3, train_loss = 0.22256482f0, test_loss = 0.22487973f0, dloss = 0.00037708879f0, seconds_per_chunk = 23.240786478, seconds_per_epoch = 3200.828148853)
[ Info: (epoch = 12, ch = 4, train_loss = 0.22227426f0, test_loss = 0.22641999f0, dloss = 0.00029055774f0, seconds_per_chunk = 23.284128892, seconds_per_epoch = 3200.828148853)
[ Info: (epoch = 12, ch = 5, train_loss = 0.2231141f0, test_loss = 0.22305325f0, dloss = -0.00083984435f0, seconds_per_chunk = 23.8

In [ ]:
#Plot losses
Plots.plot(train_losses; label="train loss", xlabel="epoch", ylabel="MSE loss", linewidth=2, yaxis=:log, ylim=[10^-1.8, 10^-1.2])
Plots.plot!(test_losses; label="test loss", linewidth=2)
Plots.plot!([ mean(train_losses[max(1,i):min(end,i+50)]) for i in 1:length(train_losses)-50 ], label="Train loss 50 ch. mov. avg.")
Plots.plot!([ mean(test_losses[max(1,i):min(end,i+50)]) for i in 1:length(test_losses)-50 ], label="Test loss 50 ch. mov. avg.")